In [ ]:
# 실습1
import numpy as np
import cv2 as cv
import sys
from google.colab import drive
from google.colab.patches import cv2_imshow

drive.mount('/gdrive')
drive_path = "/gdrive/My Drive/yolo/Recognition/"

def construct_yolo_v3():
    f = open(drive_path + 'coco_names.txt', 'r')
    class_names = [line.strip() for line in f.readlines()]

    model = cv.dnn.readNet(drive_path +'yolov3.weights', drive_path +'yolov3.cfg')
    layer_names = model.getLayerNames()
    out_layers = [layer_names[i - 1] for i in model.getUnconnectedOutLayers()]

    return model, out_layers, class_names

def yolo_detect(img, yolo_model, out_layers):
    height, width = img.shape[0], img.shape[1]
    test_img = cv.dnn.blobFromImage(img, 1.0 / 256, (448, 448), (0, 0, 0), swapRB=True)

    yolo_model.setInput(test_img)
    output3 = yolo_model.forward(out_layers)

    box, conf, id = [], [], []

    for output in output3:
        for vec85 in output:
            scores = vec85[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                centerx, centery = int(vec85[0] * width), int(vec85[1] * height)
                w, h = int(vec85[2] * width), int(vec85[3] * height)
                x, y = int(centerx - w / 2), int(centery - h / 2)
                box.append([x, y, x + w, y + h])
                conf.append(float(confidence))
                id.append(class_id)

    ind = cv.dnn.NMSBoxes(box, conf, 0.5, 0.4)
    objects = [box[i] + [conf[i]] + [id[i]] for i in range(len(box)) if i in ind]

    return objects

model, out_layers, class_names = construct_yolo_v3()
colors = np.random.uniform(0, 255, size=(len(class_names), 3))

img = cv.imread(drive_path +'soccer.jpg')
if img is None: sys.exit("파일이 없습니다.")

res = yolo_detect(img, model, out_layers)

for i in range(len(res)):
    x1, y1, x2, y2, confidence, id = res[i]
    text = str(class_names[id]) + ':%.3f' % confidence
    cv.rectangle(img, (x1, y1), (x2, y2), colors[id], 2)
    cv.putText(img, text, (x1, y1 + 30), cv.FONT_HERSHEY_PLAIN, 1.5, colors[id], 2)

cv2_imshow(img)
cv.waitKey()
cv.destroyAllWindows()


In [ ]:
#실습 4번

from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras import layers
import os
import random
import cv2 as cv
from google.colab import drive
from google.colab.patches import cv2_imshow

drive.mount('/gdrive')
drive_path = "/gdrive/My Drive/dataset/datasets2/oxford_pets/"


input_dir = drive_path + "/images/images"
target_dir = drive_path + "/annotations/annotations/trimaps"
img_siz=(160,160)
n_class=3
batch_siz=32

img_paths = sorted([os.path.join(input_dir,f) for f in os.listdir(input_dir) if f.endswith('.jpg')])
label_paths = sorted([os.path.join(target_dir,f) for f in os.listdir(target_dir) if f.endswith('.png') and not f.startswith('.')])

class OxfordPets(keras.utils.Sequence):
    def __init__(self,batch_size,img_size,img_paths,label_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_paths = img_paths
        self.label_paths = label_paths

    def __len__(self):
        return len(self.label_paths)//self.batch_size

    def __getitem__(self,idx):
        i = idx*self.batch_size
        batch_img_paths = self.img_paths[i:i+self.batch_size]
        batch_label_paths = self.label_paths[i:i+self.batch_size]
        x= np.zeros((self.batch_size,)+self.img_size+(3,),dtype="float32")
        for j,path in enumerate(batch_img_paths):
            img = load_img(path,target_size = self.img_size)
            x[j]=img
        y=np.zeros((self.batch_size,)+self.img_size+(1,),dtype = "uint8")
        for j,path in enumerate(batch_label_paths):
            img = load_img(path,target_size = self.img_size,color_mode = "grayscale")
            y[j]=np.expand_dims(img,2)
            y[j]-=1
        return x,y

def make_model(img_size,num_classes):
    inputs = keras.Input(shape = img_size+(3,))


    # U-net의 다운 샘플링(축소경로)
    x = layers.Conv2D(32,3,strides=2,padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    previous_block_activation = x

    for filters in [64,128,256]:
        x = layers.Activation('relu')(x)
        x = layers.SeparableConv2D(filters,3,padding = 'same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.SeparableConv2D(filters,3,padding = 'same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D(3,strides=2,padding = 'same')(x)
        residual = layers.Conv2D(filters,1,strides=2,padding='same')(previous_block_activation)
        x = layers.add([x,residual])
        previous_block_activation = x

    # U-net 업 샘플링(확대 경로)
    for filters in [256 ,128, 64 ,32]:
        x = layers.Activation('relu')(x)
        x = layers.Conv2DTranspose(filters,3,padding = 'same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.Conv2DTranspose(filters,3,padding = 'same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.UpSampling2D(2)(x)
        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters,1,padding = 'same')(residual)
        x = layers.add([x,residual])
        previous_block_activation = x

    outputs = layers.Conv2D(num_classes,3,activation = 'softmax',padding = 'same')(x)
    model = keras.Model(inputs, outputs)
    return model

model = make_model(img_siz,n_class)

random.Random(1).shuffle(img_paths)
random.Random(1).shuffle(label_paths)
test_samples = int(len(img_paths)*0.1)
train_img_paths = img_paths[:-test_samples]
train_label_paths = label_paths[:-test_samples]
test_img_paths = img_paths[-test_samples:]
test_label_paths = label_paths[-test_samples:]

train_gen = OxfordPets(batch_siz,img_siz,train_img_paths,train_label_paths)
test_gen = OxfordPets(batch_siz,img_siz,test_img_paths,test_label_paths)

model.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy',metrics = ['accuracy'])
cb = [keras.callbacks.ModelCheckpoint('oxford_seg.h5',save_best_only=True)]
model.fit(train_gen,epochs = 20,validation_data = test_gen,callbacks = cb)

preds = model.predict(test_gen)

cv2_imshow(cv.imread(test_img_paths[0]))
cv2_imshow(cv.imread(test_label_paths[0])*64)
cv2_imshow(preds[0])

cv.waitKey()
cv.destroyAllWindows()